# **RA1: quarterly panel data**

Dong Gyun Ko <br/>
last updated: september 13, 2022 <br/>

In [1]:
# import libraries

import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader as pdr
import pandas_datareader.data as web
from pandas_datareader import wb
from pandas_datareader.data import DataReader
import requests # python 3.6

import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# set the working directory

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/RA1_data')

In [3]:
# set the pandas display option

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# pd.options.display.max_rows = 100
# pd.options.display.max_columns = 100

## **1. panel data**

### **1.1. python code** <br/>

In [4]:
# import the pooled data

df_q_pooled = pd.read_excel('df_q_pooled.xlsx').drop(columns=['Unnamed: 0'])
df_q_pooled['quarter'] = pd.to_datetime(df_q_pooled['quarter'])
df_q_pooled['quarter'] = df_q_pooled['quarter'].dt.to_period('Q')

In [5]:
# generate the advanced country dummy variable

df_q_pooled['dum_advanced'] = np.nan

advanced_country_list = ['KOR', 'AUT', 'BEL', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC',
                         'ISL', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'NOR', 'PRT',
                         'SMR', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'GBR', 'CYP', 'ISR', 'JPN',
                         'SGP', 'CAN', 'USA', 'AUS', 'NZL', 'PRI', 'MAC', 'HKG']

for i in range(0, len(advanced_country_list)):
  
    globals()['cond_advanced_country_{}'.format(i+1)] = (df_q_pooled['isocode'] == advanced_country_list[i])
    df_q_pooled.loc[globals()['cond_advanced_country_{}'.format(i+1)], 'dum_advanced'] = 1

In [6]:
# quarterly pooled dataset

df_q_pooled = df_q_pooled.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_q_pooled

,country,isocode,quarter,oecd_q_ggexp,oecd_q_gginv,oecd_q_rgdp_dcu,oecd_q_rgdp_usd,oecd_q_ca,oecd_q_stir,bis_q_ggdebt_m,...,imf_q_cpi,imf_q_gdpd,imf_q_neer,imf_q_reer,imf_q_ca,imf_q_stir_mm,imf_q_stir_3mtb,imf_q_stir_mp,fred_q_ggexp,dum_advanced
0,Aruba,ABW,1913Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,1913Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,ABW,1913Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,1913Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,1914Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95041,Zimbabwe,ZWE,2021Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4984.783547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95042,Zimbabwe,ZWE,2021Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5520.552280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95043,Zimbabwe,ZWE,2021Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6497.511865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95044,Zimbabwe,ZWE,2022Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# generate the count matrix for quarterly pooled data

count_N_df_q_pooled = df_q_pooled.groupby(df_q_pooled['isocode']).count()
count_T_df_q_pooled = df_q_pooled.groupby(df_q_pooled['quarter']).count()

In [8]:
# filter (N)

# ggdebt
condN1 = (count_N_df_q_pooled['bis_q_ggdebt_m'] == 0)
condN2 = (count_N_df_q_pooled['bis_q_ggdebt_n'] == 0)
condN3 = (count_N_df_q_pooled['wb_q_ggdebt_d2'] == 0)
condN4 = (count_N_df_q_pooled['eustat_q_ggdebt'] == 0)
condN5 = (count_N_df_q_pooled['imf_q_ggdebt'] == 0)

# ggexp
condN6 = (count_N_df_q_pooled['oecd_q_ggexp'] == 0)
condN7 = (count_N_df_q_pooled['eustat_q_ggexp'] == 0)
condN8 = (count_N_df_q_pooled['imf_q_ggexp'] == 0)
condN9 = (count_N_df_q_pooled['fred_q_ggexp'] == 0)

count_N_df_q_pooled = count_N_df_q_pooled.loc[(~condN1 | ~condN2 | ~condN3 | ~condN4 | ~condN5) & (~condN6 | ~condN7 | ~condN8 | ~condN9)]
count_N_df_q_pooled = count_N_df_q_pooled.reset_index()

print('countries:', count_N_df_q_pooled['isocode'].unique().shape[0])

countries: 50


In [9]:
# filter (T)

# ggdebt
condT1 = (count_T_df_q_pooled['bis_q_ggdebt_m'] == 0)
condT2 = (count_T_df_q_pooled['bis_q_ggdebt_n'] == 0)
condT3 = (count_T_df_q_pooled['wb_q_ggdebt_d2'] == 0)
condT4 = (count_T_df_q_pooled['eustat_q_ggdebt'] == 0)
condT5 = (count_N_df_q_pooled['imf_q_ggdebt'] == 0)

# ggexp
condT6 = (count_T_df_q_pooled['oecd_q_ggexp'] == 0)
condT7 = (count_T_df_q_pooled['eustat_q_ggexp'] == 0)
condT8 = (count_T_df_q_pooled['imf_q_ggexp'] == 0)
condT9 = (count_T_df_q_pooled['fred_q_ggexp'] == 0)

count_T_df_q_pooled = count_T_df_q_pooled.loc[(~condT1 | ~condT2 | ~condT3 | ~condT4 | ~condT5) & (~condT6 | ~condT7 | ~condT8 | ~condT9)]
count_T_df_q_pooled = count_T_df_q_pooled.reset_index()

print('start:', count_T_df_q_pooled['quarter'].unique().min())
print('end:', count_T_df_q_pooled['quarter'].unique().max())

start: 1955Q1
end: 2022Q1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py:4024: RuntimeWarning: '<' not supported between instances of 'int' and 'Period', sort order is undefined for incomparable objects
  join_index = self.union(other)


In [10]:
# generate the panel data

# select the T
cond_T = (df_q_pooled['quarter'].dt.year >= 1955)

df_q_panel = df_q_pooled.loc[cond_T]

# select the N
print(count_N_df_q_pooled['isocode'].unique().shape[0])
isocode_list = count_N_df_q_pooled['isocode'].unique()

for i in range(0, count_N_df_q_pooled['isocode'].unique().shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_q_pooled['isocode'] == isocode_list[i])

df_q_panel = df_q_panel.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

50


In [11]:
# select the columns

df_q_panel = df_q_panel[['country', 'isocode', 'quarter', 'dum_advanced', 
                         'oecd_q_ggexp', 'oecd_q_rgdp_dcu', 'oecd_q_rgdp_usd', 'oecd_q_ca', 'oecd_q_stir',
                         'bis_q_ggdebt_m', 'bis_q_ggdebt_n', 'bis_q_cbpr', 'bis_q_reer_b', 'bis_q_reer_n',
                         'wb_q_ggdebt_d1', 'wb_q_ggdebt_d2', 'wb_q_ggdebt_d3', 'wb_q_ggdebt_d4',
                         'eustat_q_ggdebt', 'eustat_q_ggexp',
                         'imf_q_ggexp', 'imf_q_rgdp', 'imf_q_cpi', 'imf_q_gdpd', 'imf_q_reer', 'imf_q_ca', 'imf_q_stir_mm',
                         'fred_q_ggexp']]

In [12]:
# generate the variables from the pooled data source

df_q_panel['pds_q_ggdebt'] = df_q_panel['bis_q_ggdebt_n']
df_q_panel['pds_q_ggexp'] = df_q_panel['imf_q_ggexp']
df_q_panel['pds_q_rgdp'] = df_q_panel['imf_q_rgdp']
df_q_panel['pds_q_reer'] = df_q_panel['bis_q_reer_b']
df_q_panel['pds_q_ca'] = df_q_panel['oecd_q_ca']
df_q_panel['pds_q_stir'] = df_q_panel['oecd_q_stir']

In [13]:
# replace the value of pds variables

# ggdebt (1)
bis_q_ggdebt_m_list = ['CHL', 'KOR']

for i in range(0, len(bis_q_ggdebt_m_list)):
  
    globals()['cond_bis_q_ggdebt_m_{}'.format(i+1)] = (df_q_panel['isocode'] == bis_q_ggdebt_m_list[i])
    df_q_panel.loc[globals()['cond_bis_q_ggdebt_m_{}'.format(i+1)], 'pds_q_ggdebt'] = df_q_panel['bis_q_ggdebt_m']

# ggdebt (2)
cond_wb_q_ggdebt_d1 = (df_q_panel['isocode'] == 'CYP')
df_q_panel.loc[cond_wb_q_ggdebt_d1, 'pds_q_ggdebt'] = df_q_panel['wb_q_ggdebt_d1']

# ggdebt (3)
wb_q_ggdebt_d2_list = ['AUT', 'BGR', 'BRA', 'CHE', 'CZE', 'DEU', 'DNK', 'EST', 'FIN', 'FRA', 
                       'IDN', 'IRL', 'ITA', 'LTU', 'LUX', 'LVA', 'MLT', 'NLD', 'NOR', 'POL', 
                       'PRT', 'ROU', 'SVK', 'SWE']

for i in range(0, len(wb_q_ggdebt_d2_list)):
  
    globals()['cond_wb_q_ggdebt_d2_{}'.format(i+1)] = (df_q_panel['isocode'] == wb_q_ggdebt_d2_list[i])
    df_q_panel.loc[globals()['cond_wb_q_ggdebt_d2_{}'.format(i+1)], 'pds_q_ggdebt'] = df_q_panel['wb_q_ggdebt_d2']

# ggdebt (4)
eustat_q_ggdebt_list = ['HRV', 'SVN']

for i in range(0, len(eustat_q_ggdebt_list)):
  
    globals()['cond_eustat_q_ggdebt_{}'.format(i+1)] = (df_q_panel['isocode'] == eustat_q_ggdebt_list[i])
    df_q_panel.loc[globals()['cond_eustat_q_ggdebt_{}'.format(i+1)], 'pds_q_ggdebt'] = df_q_panel['eustat_q_ggdebt']

# ggexp
cond_fred_q_ggexp = (df_q_panel['isocode'] == 'EST')
df_q_panel.loc[cond_fred_q_ggexp, 'pds_q_ggexp'] = df_q_panel['fred_q_ggexp']

# reer
imf_q_reer_list = ['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL', 'CYP', 
                   'CZE', 'DEU', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'HUN', 'IRL', 
                   'ISR', 'ITA', 'JPN', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'NOR', 'NZL', 
                   'POL', 'PRT', 'ROU', 'RUS', 'SGP', 'SVK', 'SWE', 'USA', 'ZAF']

for i in range(0, len(imf_q_reer_list)):
  
    globals()['cond_imf_q_reer_{}'.format(i+1)] = (df_q_panel['isocode'] == imf_q_reer_list[i])
    df_q_panel.loc[globals()['cond_imf_q_reer_{}'.format(i+1)], 'pds_q_reer'] = df_q_panel['imf_q_reer']

In [14]:
# quarterly panel data

df_q_panel = df_q_panel.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_q_panel

,country,isocode,quarter,dum_advanced,oecd_q_ggexp,oecd_q_rgdp_dcu,oecd_q_rgdp_usd,oecd_q_ca,oecd_q_stir,bis_q_ggdebt_m,...,imf_q_reer,imf_q_ca,imf_q_stir_mm,fred_q_ggexp,pds_q_ggdebt,pds_q_ggexp,pds_q_rgdp,pds_q_reer,pds_q_ca,pds_q_stir
0,Argentina,ARG,1955Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Argentina,ARG,1955Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Argentina,ARG,1955Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,ARG,1955Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,ARG,1956Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,South Africa,ZAF,2021Q2,NaN,NaN,NaN,NaN,NaN,8.600000,NaN,...,80.196367,7.139185e+09,3.510000,222850.4,70.1,2.228500e+11,1.136380e+12,80.196367,NaN,8.600000
13496,South Africa,ZAF,2021Q3,NaN,NaN,NaN,NaN,NaN,9.156667,NaN,...,78.671305,2.482808e+09,3.530000,224008.7,70.3,2.240090e+11,1.115900e+12,78.671305,NaN,9.156667
13497,South Africa,ZAF,2021Q4,NaN,NaN,NaN,NaN,NaN,10.156667,NaN,...,75.720268,3.494472e+09,3.626667,224479.8,70.9,2.244800e+11,1.131190e+12,75.720268,NaN,10.156667
13498,South Africa,ZAF,2022Q1,NaN,NaN,NaN,NaN,NaN,10.263333,NaN,...,76.993295,6.237743e+08,3.956667,226828.7,NaN,2.268290e+11,1.153130e+12,76.993295,NaN,10.263333


### **1.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|$Dummy$|||||||||
|1|advanced country dummy|dum_advanced|-|-|-|-|-|-|
|$OECD$|||||||||
|1|government final consumption expenditure, volume|oecd_q_ggexp|DCU, base year|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|2|gross domestic product, volume, market prices|oecd_q_rgdp_dcu|DCU, base year|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|3|gross domestic product, volume in USD, constant exchange rates|oecd_q_rgdp_usd|USD, 2015|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|4|current account balance as a percentage of GDP|oecd_q_ca|% of GDP|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|5|short-term interest rate|oecd_q_stir|% per annum|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|$BIS$|||||||||
|1|total credit to the government sector at market value (core debt)|bis_q_ggdebt_m|% of GDP|quarterly|CRE|-|market value|https://stats.bis.org/statx/srs/table/f5.1|
|2|total credit to the government sector at nominal value (core debt)|bis_q_ggdebt_n|% of GDP|quarterly|CRE|-|nominal value|https://stats.bis.org/statx/srs/table/f5.4|
|3|central bank policy rates|bis_q_cbpr|%|quarterly|CBP|-|-|https://stats.bis.org/statx/srs/table/l1|
|4|real effective exchange rate, broad(60 economies) indicies|bis_q_reer_b|2010=100|quarterly|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|
|5|real effective exchange rate, narrow(27 economies) indicies|bis_q_reer_n|2010=100|quarterly|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|
|$WB$|||||||||
|1|Gross PSD, General Gov.-D1, All maturities, Debt securities + loans, Nominal Value, % of GDP|wb_q_ggdebt_d1|% of GDP|quarterly|DP.DOD.DLD1.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|2|Gross PSD, General Gov.-D2, All maturities, D1+ SDRs + currency and deposits, Nominal Value, % of GDP|wb_q_ggdebt_d2|% of GDP|quarterly|DP.DOD.DLD2.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|3|Gross PSD, General Gov.-D3, All maturities, D2+other accounts payable, Nominal Value, % of GDP|wb_q_ggdebt_d3|% of GDP|quarterly|DP.DOD.DLD3.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|4|Gross PSD, General Gov.-D4, All maturities, D3+insurance, pensions, and standardized guarantees, Nominal Value, % of GDP|wb_q_ggdebt_d4|% of GDP|quarterly|DP.DOD.DLD2.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|$Eurostat$|||||||||
|1|government consolidated gross debt [S13: general government]|eustat_q_ggdebt|% of GDP|quarterly|GOV_10Q_GGDEBT [GD]|-|PC_GDP|https://ec.europa.eu/eurostat/databrowser/view/GOV_10Q_GGDEBT__custom_3260679/default/table?lang=en|
|2|final consumption expenditure [S13: general government]|eustat_q_ggexp|% of GDP|quarterly|GOV_10Q_GGNFA [P3]|-|PC_GDP / NSA|https://ec.europa.eu/eurostat/databrowser/view/GOV_10Q_GGNFA__custom_3263185/default/table?lang=en|
|$IMF$|||||||||
|1|general government final consumption expenditure, real|imf_q_ggexp|DCU|quarterly|NCGG_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|2|gross domestic product, real|imf_q_rgdp|DCU|quarterly|NGDP_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|3|prices, consumer price index, all items|imf_q_cpi|index|quarterly|PCPI_IX|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|4|gross domestic product, deflator|imf_q_gdpd|index|quarterly|NGDP_D_SA_IX|-|SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|5|exchange rates, real effective exchange rate based on consumer price index|imf_q_reer|index|quarterly|EREER_IX|-|real|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|6|supplementary items, current account, net (excluding exceptional financing)|imf_q_ca|USD|quarterly|BCAXF_BP6_USD|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|7|financial, interest rates, money market|imf_q_stir_mm|percent per annum|quarterly|FIMM_PA|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|$FRED$|||||||||
|1|real general government final consumption expenditure|fred_q_ggexp|DCU|quarterly|NCGGRSAXDC + isocode [alpha-2] + Q|-|real / SA|https://fred.stlouisfed.org/searchresults?st=real+general+government+final+consumption+expenditure|
|$PDS$|||||||||
|1|general government debt|pds_q_ggdebt|% of GDP|quarterly|-|-|-|pooled data source|
|2|general government consumption, real|pds_q_ggexp|DCU|quarterly|-|-|real / SA|pooled data source|
|3|gross domestic product, real|pds_q_rgdp|DCU|quarterly|-|-|real / SA|pooled data source|
|4|real effective exchange rate|pds_q_reer|index|quarterly|-|-|real|pooled data source|
|5|current account balance|pds_q_ca|% of GDP|quarterly|-|-|-|pooled data source|
|6|short-term interest rate|pds_q_stir|% per annum|quarterly|-|-|-|pooled data source|

### **1.3. data summary**

In [15]:
# number of countries & sample period

print('number of countries:', df_q_panel['isocode'].unique().shape[0])
print(df_q_panel['isocode'].unique())
print('start:', np.min(df_q_panel['quarter'].unique()))
print('end:', np.max(df_q_panel['quarter'].unique()))

number of countries: 50
['ARG' 'AUS' 'AUT' 'BEL' 'BGR' 'BRA' 'CAN' 'CHE' 'CHL' 'COL' 'CYP' 'CZE'
 'DEU' 'DNK' 'ESP' 'EST' 'FIN' 'FRA' 'GBR' 'GRC' 'HKG' 'HRV' 'HUN' 'IDN'
 'IND' 'IRL' 'ISR' 'ITA' 'JPN' 'KOR' 'LTU' 'LUX' 'LVA' 'MEX' 'MLT' 'NLD'
 'NOR' 'NZL' 'POL' 'PRT' 'ROU' 'RUS' 'SGP' 'SVK' 'SVN' 'SWE' 'THA' 'TUR'
 'USA' 'ZAF']
start: 1955Q1
end: 2022Q2


In [16]:
# non-null count by each variables

df_q_panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13500 entries, 0 to 13499
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype        
---  ------           --------------  -----        
 0   country          13500 non-null  object       
 1   isocode          13500 non-null  object       
 2   quarter          13500 non-null  period[Q-DEC]
 3   dum_advanced     9180 non-null   float64      
 4   oecd_q_ggexp     4340 non-null   float64      
 5   oecd_q_rgdp_dcu  4456 non-null   float64      
 6   oecd_q_rgdp_usd  4456 non-null   float64      
 7   oecd_q_ca        3534 non-null   float64      
 8   oecd_q_stir      4180 non-null   float64      
 9   bis_q_ggdebt_m   3164 non-null   float64      
 10  bis_q_ggdebt_n   4394 non-null   float64      
 11  bis_q_cbpr       4583 non-null   object       
 12  bis_q_reer_b     5700 non-null   float64      
 13  bis_q_reer_n     5850 non-null   float64      
 14  wb_q_ggdebt_d1   3141 non-null   float64      
 15  wb

In [17]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_q_panel.describe(), 2)

,dum_advanced,oecd_q_ggexp,oecd_q_rgdp_dcu,oecd_q_rgdp_usd,oecd_q_ca,oecd_q_stir,bis_q_ggdebt_m,bis_q_ggdebt_n,bis_q_reer_b,bis_q_reer_n,...,imf_q_reer,imf_q_ca,imf_q_stir_mm,fred_q_ggexp,pds_q_ggdebt,pds_q_ggexp,pds_q_rgdp,pds_q_reer,pds_q_ca,pds_q_stir
count,9180.00,4340.00,4456.00,4456.00,3534.00,4180.00,3164.00,4394.00,5700.00,5850.00,...,6439.00,7445.00,5644.00,2406.00,5925.00,6174.00,6369.00,7591.00,3534.00,4180.00
mean,1.00,6542472575848.33,45445265002657.84,927588342684.50,-0.63,8.65,65.37,60.35,96.15,100.15,...,96.94,-811218270.65,35.02,9786714.16,51.23,4341086182192.95,38993486399776.12,97.43,-0.63,8.65
std,0.00,21341904505376.79,147422616429373.91,1994209164892.58,4.30,6.34,39.61,34.97,18.08,17.21,...,16.13,18442322185.77,894.91,32293790.33,36.74,20844113378891.70,235020963922364.12,19.10,4.30,6.34
min,1.00,1365709967.41,8075649110.82,8956691869.00,-20.14,0.05,4.60,3.60,41.04,45.28,...,32.42,-255473000000.00,-0.68,9907.00,0.00,29962.28,1293591273.00,32.42,-20.14,0.05
25%,1.00,36126826394.71,187550515950.61,126901687722.33,-2.93,4.40,38.60,35.00,89.56,90.45,...,90.02,-1747000000.00,2.67,53192.70,26.90,10303425000.00,54868400000.00,89.78,-2.93,4.40
50%,1.00,158832000000.00,760965321672.18,268975028885.86,-0.63,7.00,57.10,53.20,97.58,98.45,...,97.97,-206755175.70,5.50,83369.98,44.48,49381450000.00,267076000000.00,97.94,-0.63,7.00
75%,1.00,460097500000.00,2316546626302.49,828358210890.16,1.71,11.46,84.10,75.80,102.43,106.97,...,103.71,1323288969.00,10.53,415719.32,69.80,143672750000.00,776728000000.00,103.85,1.71,11.46
max,1.00,180434000000000.00,1332547200000000.00,16533342932403.30,18.93,57.84,238.20,226.90,281.92,248.02,...,203.52,94237700000.00,61846.25,230259300.00,226.90,230259000000000.00,2920000000000000.00,281.92,18.93,57.84


## **2. count the data**

In [18]:
# generate the count matrix for each country's quarterly data

for i in range(0, df_q_panel['isocode'].unique().shape[0]):
  
    globals()['cond_isocode_{}'.format(i+1)] = (df_q_panel['isocode'] == df_q_panel['isocode'].unique()[i])
    globals()['df_isocode_{}'.format(i+1)] = df_q_panel.loc[globals()['cond_isocode_{}'.format(i+1)]]
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['df_isocode_{}'.format(i+1)].groupby(globals()['df_isocode_{}'.format(i+1)]['quarter']).count()

    globals()['cond_pds_q_ggdebt_{}'.format(i+1)] = (globals()['count_T_df_isocode_{}'.format(i+1)]['pds_q_ggdebt'] == 0)
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['count_T_df_isocode_{}'.format(i+1)].loc[~globals()['cond_pds_q_ggdebt_{}'.format(i+1)]]
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['count_T_df_isocode_{}'.format(i+1)].reset_index()

    print('country', df_q_panel['country'].unique()[i])
    print('isocode', df_q_panel['isocode'].unique()[i])
    print('start:', globals()['count_T_df_isocode_{}'.format(i+1)]['quarter'].unique().min())
    print('end:', globals()['count_T_df_isocode_{}'.format(i+1)]['quarter'].unique().max())
    print('obs.:', globals()['count_T_df_isocode_{}'.format(i+1)]['pds_q_ggdebt'].count())
    print('des. stat.', globals()['df_isocode_{}'.format(i+1)]['pds_q_ggdebt'].describe())
    print('')

country Argentina
isocode ARG
start: 1992Q4
end: 2021Q4
obs.: 117
des. stat. count   117.00
mean     63.06
std      31.71
min      25.80
25%      40.50
50%      54.80
75%      83.70
max     154.90
Name: pds_q_ggdebt, dtype: float64

country Australia
isocode AUS
start: 1988Q2
end: 2021Q4
obs.: 135
des. stat. count   135.00
mean     24.96
std      11.40
min       8.30
25%      15.30
50%      24.40
75%      32.90
max      54.60
Name: pds_q_ggdebt, dtype: float64

country Austria
isocode AUT
start: 1995Q4
end: 2022Q1
obs.: 106
des. stat. count   106.00
mean     64.12
std      28.80
min       0.00
25%      67.64
50%      73.27
75%      82.32
max      87.11
Name: pds_q_ggdebt, dtype: float64

country Belgium
isocode BEL
start: 1995Q1
end: 2021Q4
obs.: 108
des. stat. count   108.00
mean    108.67
std      11.18
min      87.30
25%     102.30
50%     106.90
75%     112.27
max     135.70
Name: pds_q_ggdebt, dtype: float64

country Bulgaria
isocode BGR
start: 1995Q1
end: 2022Q1
obs.: 109
des. st

In [19]:
# generate the count matrix for each country's quarterly data

for i in range(0, df_q_panel['isocode'].unique().shape[0]):
  
    globals()['cond_isocode_{}'.format(i+1)] = (df_q_panel['isocode'] == df_q_panel['isocode'].unique()[i])
    globals()['df_isocode_{}'.format(i+1)] = df_q_panel.loc[globals()['cond_isocode_{}'.format(i+1)]]
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['df_isocode_{}'.format(i+1)].groupby(globals()['df_isocode_{}'.format(i+1)]['quarter']).count()

    globals()['cond_pds_q_ggexp_{}'.format(i+1)] = (globals()['count_T_df_isocode_{}'.format(i+1)]['pds_q_ggexp'] == 0)
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['count_T_df_isocode_{}'.format(i+1)].loc[~globals()['cond_pds_q_ggexp_{}'.format(i+1)]]
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['count_T_df_isocode_{}'.format(i+1)].reset_index()

    print('country', df_q_panel['country'].unique()[i])
    print('isocode', df_q_panel['isocode'].unique()[i])
    print('start:', globals()['count_T_df_isocode_{}'.format(i+1)]['quarter'].unique().min())
    print('end:', globals()['count_T_df_isocode_{}'.format(i+1)]['quarter'].unique().max())
    print('obs.:', globals()['count_T_df_isocode_{}'.format(i+1)]['pds_q_ggexp'].count())
    print('des. stat.', globals()['df_isocode_{}'.format(i+1)]['pds_q_ggexp'].describe())
    print('')

country Argentina
isocode ARG
start: 2004Q1
end: 2022Q1
obs.: 73
des. stat. count            73.00
mean    20454731506.85
std      3576286378.43
min     13129300000.00
25%     17557800000.00
50%     21715700000.00
75%     23499600000.00
max     24895700000.00
Name: pds_q_ggexp, dtype: float64

country Australia
isocode AUS
start: 1959Q3
end: 2022Q1
obs.: 251
des. stat. count            251.00
mean     45312653386.45
std      27144011372.74
min       9560000000.00
25%      23252500000.00
50%      40164000000.00
75%      64889000000.00
max     118314000000.00
Name: pds_q_ggexp, dtype: float64

country Austria
isocode AUT
start: 1995Q1
end: 2022Q2
obs.: 110
des. stat. count           110.00
mean    14407310909.09
std      1702363946.97
min     11466700000.00
25%     12827350000.00
50%     14886750000.00
75%     15535550000.00
max     18589000000.00
Name: pds_q_ggexp, dtype: float64

country Belgium
isocode BEL
start: 1995Q1
end: 2022Q1
obs.: 109
des. stat. count           109.00
mean    2

In [20]:
# generate the count matrix for each country's quarterly data

for i in range(0, df_q_panel['isocode'].unique().shape[0]):
  
    globals()['cond_isocode_{}'.format(i+1)] = (df_q_panel['isocode'] == df_q_panel['isocode'].unique()[i])
    globals()['df_isocode_{}'.format(i+1)] = df_q_panel.loc[globals()['cond_isocode_{}'.format(i+1)]]
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['df_isocode_{}'.format(i+1)].groupby(globals()['df_isocode_{}'.format(i+1)]['quarter']).count()

    globals()['cond_pds_q_reer_{}'.format(i+1)] = (globals()['count_T_df_isocode_{}'.format(i+1)]['pds_q_reer'] == 0)
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['count_T_df_isocode_{}'.format(i+1)].loc[~globals()['cond_pds_q_reer_{}'.format(i+1)]]
    globals()['count_T_df_isocode_{}'.format(i+1)] = globals()['count_T_df_isocode_{}'.format(i+1)].reset_index()

    print('country', df_q_panel['country'].unique()[i])
    print('isocode', df_q_panel['isocode'].unique()[i])
    print('start:', globals()['count_T_df_isocode_{}'.format(i+1)]['quarter'].unique().min())
    print('end:', globals()['count_T_df_isocode_{}'.format(i+1)]['quarter'].unique().max())
    print('obs.:', globals()['count_T_df_isocode_{}'.format(i+1)]['pds_q_reer'].count())
    print('des. stat.', globals()['df_isocode_{}'.format(i+1)]['pds_q_reer'].describe())
    print('')

country Argentina
isocode ARG
start: 1994Q1
end: 2022Q2
obs.: 114
des. stat. count   114.00
mean    135.86
std      73.99
min      52.01
25%      78.24
50%     107.71
75%     227.29
max     281.92
Name: pds_q_reer, dtype: float64

country Australia
isocode AUS
start: 1980Q1
end: 2022Q2
obs.: 170
des. stat. count   170.00
mean     86.70
std      11.02
min      67.12
25%      77.04
50%      87.49
75%      94.29
max     110.51
Name: pds_q_reer, dtype: float64

country Austria
isocode AUT
start: 1979Q1
end: 2022Q2
obs.: 174
des. stat. count   174.00
mean    101.44
std       3.40
min      93.93
25%      99.08
50%     101.72
75%     103.30
max     111.76
Name: pds_q_reer, dtype: float64

country Belgium
isocode BEL
start: 1979Q1
end: 2022Q2
obs.: 174
des. stat. count   174.00
mean    100.66
std       4.82
min      91.85
25%      98.07
50%     100.62
75%     102.50
max     120.90
Name: pds_q_reer, dtype: float64

country Bulgaria
isocode BGR
start: 1993Q1
end: 2022Q2
obs.: 118
des. stat. coun

## **3. export the data**

In [21]:
# generate the count matrix for quarterly panel data

count_N_df_q_panel = df_q_panel.groupby(df_q_panel['isocode']).count()
count_T_df_q_panel = df_q_panel.groupby(df_q_panel['quarter']).count()

In [22]:
# count matrix for quarterly panel data

count_N_df_q_panel.to_excel(excel_writer='count_N_df_q_panel.xlsx')
count_T_df_q_panel.to_excel(excel_writer='count_T_df_q_panel.xlsx')

In [23]:
# quarterly panel data

df_q_panel.to_excel(excel_writer='df_q_panel.xlsx')

## **4. references**

* https://data.oecd.org/ <br/>
* https://stats.bis.org/#ppq=XRU_D_24D;pv=1,2~4~1,0,0~both <br/>
* https://databank.worldbank.org/source/world-development-indicators/preview/on <br/>
* https://wits.worldbank.org/wits/wits/witshelp/content/codes/country_codes.htm <br/>
* https://ec.europa.eu/eurostat/en/web/main/data/database <br/>
* https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1409151240976 <br/>
* https://fred.stlouisfed.org/ <br/>